In [56]:
import pandas as pd
from ortools.linear_solver import pywraplp
import numpy as np

dataset = pd.read_csv("../Downloads/truc/faces.utf8.csv")
dataset = dataset.values
briefs = pd.read_csv("../Downloads/truc/briefs.utf8.csv")
briefs = briefs.values
result = [[]]

def get_week(week, data):
    array = []
    for i in range(0, len(data)):
        if data[i][0] == week:
            array.append(data[i])
    return (array)

dict_col = {}
dict_col["25_49csp"] = 12
dict_col["cspp"] = 15
dict_col["urb"] = 18
dict_col["all"] = 14
dict_col["f25_49rda"] = 17
dict_col["f25_49csp"] = 16
dict_col["15_24"] = 10
dict_col["25_49"] = 11
dict_col["25_49urb"] = 13

def get_max_week():
    return (briefs[len(briefs)-1][0])

def solve_week(week_nb, result):
    brief = [[]]
    full_week = get_week(week_nb, briefs)
    data = dataset.tolist()
    for k in range (0, len(full_week)):
        brief[0] = full_week[k]
        solver = pywraplp.Solver('SolveStigler', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

        faces = [[]] * len(data)
        objective = solver.Objective()
        for i in range(0, len(data)):
            faces[i] = solver.NumVar(0, 1, data[i][0])
            objective.SetCoefficient(faces[i], 1)
        objective.SetMinimization()

        constraints = [0] * len(brief)
        for i in range(0, len(brief)):
            constraints[i] = solver.Constraint(brief[i][3], solver.infinity())
            for j in range(0, len(data)):
                contactkey = dict_col[brief[i][2]]
                constraints[i].SetCoefficient(faces[j], data[j][contactkey])
                

        status = solver.Solve()
        ref = []
        if status == solver.OPTIMAL:
            nb_faces = 0
            re = []
            indexes = []
            for i in range (0, len(data)):
                if (faces[i].solution_value() > 0):
                    nb_faces += 1
                    indexes.append(i)
                    re.append(data[i][0])
            result.append(re)
            for i in reversed(indexes):
                data.pop(i)
        else:
            if status != solver.FEASIBLE:
                result.append(ref)
    return (result)

for i in range(1, get_max_week() + 1):
    result = solve_week(i, result)
print (result)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
